In [9]:
%matplotlib inline
import xarray as xr
import cftime
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

import HS_functions
import LAI_functions

#### 1. load in LAI (GLASS, CGLS, CLM5_exp) + temperature data

In [3]:
# temperature data from air max tenoerature 
ds_TSA      = xr.open_dataset(f'ihist.e52.IHistClm50BgcCrop.f19_g17.2.HSF_apsim/Temperature/T_1990_2010.nc') 
cftime_time = ds_TSA.time.values
np_time     = np.array([np.datetime64(str(t)) for t in cftime_time])
ds_TSA      = ds_TSA.assign_coords(time=("time", np_time))

# Subtract one day from each timestamp
corrected_time = np_time - pd.Timedelta(days=1)

# Assign corrected time back to dataset
ds_TSA      = ds_TSA.assign_coords(time=("time", corrected_time))
TSA         = ds_TSA['TVDAY'] - 273.15

/glade/derecho/scratch/sroos/tmp/ipykernel_15322/268707620.py:5: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds_TSA      = ds_TSA.assign_coords(time=("time", np_time))


In [4]:
# LAI Reference
ds_ref  = xr.open_dataset('GLASS_LAI_netcdf/LAI_masked_2deg/GLASSLAI_CCImasked_10pct_2deg_1981_2014.nc')
ref_lai = ds_ref['LAI']
ref_lai = ref_lai.assign_coords(lat=TSA['lat'])
years   = sorted(set(pd.to_datetime(ref_lai['time'].values).year))

In [ ]:
# LAI CLM
f_clm       = f'ihist.e52.IHistClm50BgcCrop.f19_g17.2.HSF_exp0/ELAI_90_14.nc'
ds_clm      = xr.open_dataset(f_clm)
cftime_time     = ds_clm.time.values
np_time         = np.array([np.datetime64(str(t)) for t in cftime_time])
clm_lai         = ds_clm['ELAI']
clm_lai         = clm_lai.assign_coords(time=("time", np_time))

In [37]:
# align years
ref_lai   = ref_lai.sel(time=slice(ds_TSA.time.min(), ds_TSA.time.max()))
clm_lai   = clm_lai.sel(time=slice(ref_lai.time.min(), ref_lai.time.max()))
TSA       = TSA.sel(time=slice(ref_lai.time.min(), ref_lai.time.max()))

#### 2. define conditions for AUC and HS period

In [ ]:
# calculate heatwave periods:
HW = HS_functions.HW_mask(TSA)

##### calculate mean reference LAI

In [50]:
#go from dekads to daily data
LAI_ref_daily        = ref_lai.resample(time="1D").interpolate("linear")   #LAI_CGLS.resample(time="1D").bfill()
LAI_ref_slopes_daily = calc_cgls_slopes(LAI_ref_daily)

# # Apply a 15-day rolling mean centered on each day
LAI_ref_mean     = LAI_ref_daily.rolling(time=15, center=True, min_periods=1).construct("window")

# # Convert time to day-of-year (DOY) of a no-leap year
doy_ref              = LAI_ref_mean.time.dt.dayofyear
year_ref             = LAI_ref_mean.time.dt.year
LAI_ref_mean         = LAI_ref_mean.sel(dayofyear=LAI_mean_HS.dayofyear != 60) ! check

NameError: name 'calc_cgls_slopes' is not defined

In [ ]:
# Compute mean for each day of the year across all reference years
mean_LAI         = LAI_ref_mean.groupby(adjusted_doy).mean(dim=("window", "time")).rename({'group': 'dayofyear'})
mean_LAI_diff    = mean_LAI.diff('dayofyear')

#get slope mean LAI
mLAI_mask             = np.where(((mean_LAI_diff < 0 ) & (mean_LAI> 0.1 * mean_LAI.max()) ), 1,0)
mean_lai_filtered     = mean_LAI[:-1,:,:] * mLAI_mask
mean_LAI_slope        = mean_lai_filtered.where(mean_lai_filtered  > 0)
mean_LAI_slope_masked = slope_filter(mean_LAI_slope,8)


# get doy window for mean_LAI
#date of max slope
mean_LAI_max_date = mean_LAI.idxmax(dim="dayofyear", skipna=True)
mean_LAI_max_date_plus35 = ((mean_LAI_max_date + 35 - 1) % 365) + 1

#### count valid sequences reference LAI

In [ ]:
# count valid sequences:
# Apply function over lat/lon grid
seq_count = xr.apply_ufunc(
    LAI_functions.count_valid_sequences, 
    mean_LAI_slope_masked,  # Assuming variable name
    input_core_dims=[["dayofyear"]],
    vectorize=True
)
# Mask the entire (dayofyear, lat, lon) location where seq_count > 1
LAI_slope_single = mean_LAI_slope_masked.where(seq_count <= 1)
LAI_daily_single = LAI_daily.where(seq_count <= 1)
LAI_slope_daily = LAI_slopes_daily.where(seq_count <= 1)

#### crit 1: identify pixels with only single peaks

In [ ]:
# Group by year and apply function along the time axis
LAI_slope_single_daily = LAI_slope_daily.groupby("time.year").map(lambda x: xr.apply_ufunc(
    LAI_functions.longest_non_nan_sequence, x,
    input_core_dims=[["time"]],  # Apply along time dimension
    output_core_dims=[["time"]],  # Output same shape
    vectorize=True,  # Apply element-wise
    dask="parallelized",  # Enable parallelization if using dask
    output_dtypes=[x.dtype]  # Keep dtype same as input
))

mean_LAI_single = xr.apply_ufunc(
    LAI_functions.longest_non_nan_sequence, mean_LAI,
    input_core_dims=[["dayofyear"]],  # Apply function along 'dayofyear' axis
    output_core_dims=[["dayofyear"]],  # Maintain same dimension in output
    vectorize=True,  # Apply function element-wise across lat/lon
    dask="parallelized",  # Enable parallel execution if using dask
    output_dtypes=[mean_LAI.dtype]  # Keep data type same as input
)

#### crit 2: select maximum distance of LAImax between years

In [ ]:
### period for AUC: peak + 40 days
### HS: 10 before peak - peak+40 days

#### 3. calculate AUC

#### 4. plot random points